In [3]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
def get_coords(ra_s, dec_s):
    h_ind = ra_s.find('h')
    m_ind = ra_s.find('m')
    s_ind = ra_s.find('s')    
    h = float(ra_s[:h_ind])
    m = float(ra_s[(h_ind+1):m_ind])
    s = float(ra_s[(m_ind+1):s_ind])
    ra = h + m/60 + s/3600
    if dec_s[0] == '+':
        sign = 1
    else:
        sign = -1
    d_ind = dec_s.find('°')
    m_ind = dec_s.find('′')
    s_ind = dec_s.find('″')
    d = float(dec_s[1:d_ind])
    m = float(dec_s[(d_ind+1):m_ind])
    s = float(dec_s[(m_ind+1):s_ind])
    dec = sign*(d + m/60 + s/3600)
    return ra, dec
def get_map(constellation):
    url = f'https://en.wikipedia.org/wiki/List_of_stars_in_{constellation}' #page gets downloaded according to constellation
    r = requests.get(url)

    soup = BeautifulSoup(r.content, 'lxml')  #Here, the lxml parser is used instead of HTML parser

    tab = soup.find_all('table', attrs={'class':'wikitable sortable'})[0]   #To extract information from a wikipedia table
                               
    data = [[]]
    for i in tab.find_all('tr'):   #searching in each row of the table ( 'tr' tag stands for row)
        row = []                    #declaring empty row
        for j in i.find_all('td'):  #'td' tag stands for a cell
            row.append(j.get_text())   #add the text contents of each row to the list
        data.append(row)

    heads = []
    for i in tab.find_all('tr')[:1]:
        for j in i.find_all('th'):             #'th' tag stands for header cell
            heads.append(j.get_text().strip('\n'))

    name_ind = heads.index('Name')
    ra_ind = heads.index('RA')
    dec_ind = heads.index('Dec')
                                  
    mag_ind = heads.index('abs.mag.')
    
    name = []
    ra = []
    dec = []
    mag = []
    for i in data[2:-2]:
        name_string = i[name_ind]
        try:                                             #The code first tries to run the code inside try
            ra_string = i[ra_ind].replace('\xa0', '')
            dec_string = i[dec_ind].replace('\xa0', '')   #These are code used to format the data
            mag_string = i[mag_ind]                       
            if mag_string[0]=='−':
                mag_string = '-'+mag_string[1:]
        except:                                       #If any error gets thrown up, it will execute the code inside except
            continue
        try:
            ra_i, dec_i = get_coords(ra_string, dec_string)     #convert ra dec from string to float
        except:
            continue
        try:
            mag.append(float(mag_string))
            name.append(name_string)
            ra.append(ra_i)
            dec.append(dec_i)
        except:
            continue

    name = np.array(name)
    ra = np.array(ra)
    dec = np.array(dec)
    mag = np.array(mag)
    return name, ra, dec, mag
def get_map_with_star(constellation):           #The get_map() that does not skip stars
    url = f'https://en.wikipedia.org/wiki/List_of_stars_in_{constellation}'
    r = requests.get(url)

    soup = BeautifulSoup(r.content, 'lxml')

    tab = soup.find_all('table', attrs={'class':'wikitable sortable'})[0]

    data = [[]]
    for i in tab.find_all('tr'):
        row = []
        for j in i.find_all('td'):
            row.append(j.get_text())
        data.append(row)

    heads = []
    for i in tab.find_all('tr')[:1]:
        for j in i.find_all('th'):
            heads.append(j.get_text().strip('\n'))

    name_ind = heads.index('Name')
    ra_ind = heads.index('RA')
    dec_ind = heads.index('Dec')

    mag_ind = heads.index('vis.mag.')

    name = []
    ra = []
    dec = []
    mag = []
    for i in data[2:-2]:
        name_string = i[name_ind]
        
        if i[ra_ind] != '' and i[dec_ind] != '' and i[mag_ind] != '' and i[ra_ind] != ' ' and i[dec_ind] != ' ' and i[mag_ind] != ' ' and i[mag_ind][0] != 'n':
            ra_string = i[ra_ind].replace('\xa0', '')
            dec_string = i[dec_ind].replace('\xa0', '')
            mag_string = i[mag_ind]
            if mag_string[0]=='−':
                mag_string = '-'+mag_string[1:]
            if mag_string[0]=='~':
                mag_string = mag_string[1:]
            mag_str = mag_string[0]
            for c in mag_string[1:]:
                if ord(c) > 45 and ord(c) < 59:
                    mag_str += c
                else:
                    break
            i = mag_string.find('.')
            mag_string = mag_string[:(i+3)]
                                
            ra_i, dec_i = get_coords(ra_string, dec_string)
            mag.append(float(mag_str))
            name.append(name_string)
            ra.append(ra_i)
            dec.append(dec_i)

    name = np.array(name)
    ra = np.array(ra)
    dec = np.array(dec)
    mag = np.array(mag)
    return name, ra, dec, mag
def project(ra, dec): # Stereographic Projection. 
    theta = np.deg2rad(90-dec + dec.mean())
    phi = np.deg2rad((ra-ra.mean())*15)
    x = np.sin(theta)*np.cos(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(theta)
    X = x/(1-z)
    Y = y/(1-z)
    return X,Y
def plot(constellation, return_data = False, orientation=0, norm=50, **kwargs):
    name, ra, dec, mag = get_map_with_star(constellation)
    x, y = project(ra, dec)
    s = 10**(-mag/2.5)
    if norm!=None:
        s = s/s.max()*norm
    if 'figure' not in kwargs:
        plt.figure(figsize=(10,10))
    plt.gca().set_facecolor('k')
    if 'color' not in kwargs:
        kwargs['color'] = 'w'
    if orientation == 0:
        plt.scatter(x, y, s=s, **kwargs)
    elif orientation == 1:
        plt.scatter(-y, x, s=s, **kwargs)
    elif orientation == 2:
        plt.scatter(-x, -y, s=s, **kwargs)
    elif orientation == 3:
        plt.scatter(y, -x, s=s, **kwargs)
    plt.xticks([])
    plt.yticks([])
    if return_data:
        return name, ra, dec, mag, s
    
#plot('Draco',orientation=1)

page = requests.get("https://en.wikipedia.org/wiki/List_of_natural_satellites")
soup = BeautifulSoup(page.content, 'html.parser')

table=soup.find_all('table',attrs={'class':'wikitable sortable'})[0]
data = [[]]
for i in table.find_all('tr'):  #'tr' tag used to access rows
        row = []
        for j in i.find_all('td'):  #'td' tag used to acccess elements
            row.append(j.get_text().replace(',',''))  #commas removed to enable conversion of string to float
        data.append(row)

heads = []
for i in table.find_all('tr')[:1]:
        for j in i.find_all('th'):   #'th' used to access headers
            heads.append(j.get_text().strip('\n'))
final_data=[[]]
for moon in data:
    if moon!=[]:
        final_data.append([moon[3],moon[1],moon[4]])

final_data[0]=["Name of moon","Name of planet","Mean radius(km)"]   
#final_data
for moon in final_data[1:-1]:
    moon[2]=moon[2].split('±')[0]
#final_data

[['Name of moon', 'Name of planet', 'Mean radius(km)'],
 ['Moon', 'Earth', '1737.1'],
 ['Phobos', 'Mars', '11.1'],
 ['Deimos', 'Mars', '6.2'],
 ['Io', 'Jupiter', '1818.1'],
 ['Europa', 'Jupiter', '1560.7'],
 ['Ganymede', 'Jupiter', '2634.1'],
 ['Callisto', 'Jupiter', '2408.4'],
 ['Amalthea', 'Jupiter', '83.5'],
 ['Himalia', 'Jupiter', '67'],
 ['Elara', 'Jupiter', '43 '],
 ['Pasiphae', 'Jupiter', '30 '],
 ['Sinope', 'Jupiter', '19 '],
 ['Lysithea', 'Jupiter', '18 '],
 ['Carme', 'Jupiter', '23 '],
 ['Ananke', 'Jupiter', '14 '],
 ['Leda', 'Jupiter', '10 '],
 ['Thebe', 'Jupiter', '49.3'],
 ['Adrastea', 'Jupiter', '8.2'],
 ['Metis', 'Jupiter', '21.5'],
 ['Callirrhoe', 'Jupiter', '4.3 '],
 ['Themisto', 'Jupiter', '4 '],
 ['Megaclite', 'Jupiter', '2.7 '],
 ['Taygete', 'Jupiter', '2.5 '],
 ['Chaldene', 'Jupiter', '1.9 '],
 ['Harpalyke', 'Jupiter', '2.2 '],
 ['Kalyke', 'Jupiter', '2.6 '],
 ['Iocaste', 'Jupiter', '2.6 '],
 ['Erinome', 'Jupiter', '1.6 '],
 ['Isonoe', 'Jupiter', '1.9 '],
 ['Praxid